# <center style="font-family: consolas; font-size: 32px; font-weight: bold;">  Understanding LangChain Chains for Large Language Model Application Development
 </center>

<center style="font-family: consolas; font-size: 32px; font-weight: bold;">  Hands-On LangChain for LLMs App Development: Chains </center> 



***

One of the fundamental pillars of LangChain, as implied by its name, is the concept of "chains." These chains typically integrate a large language model (LLM) with a prompt.

Through these chain structures, you have the ability to assemble multiple building blocks, enabling the execution of a series of operations on your text or other data.

This notebook will delve into the significance of these chains, ranging from basic forms like the Simple Sequential Chain to more sophisticated variations such as the Router Chain, elucidated with practical illustrations.

#### <a id="top"></a>
# <div style="box-shadow: rgb(60, 121, 245) 0px 0px 0px 3px inset, rgb(255, 255, 255) 10px -10px 0px -3px, rgb(31, 193, 27) 10px -10px, rgb(255, 255, 255) 20px -20px 0px -3px, rgb(255, 217, 19) 20px -20px, rgb(255, 255, 255) 30px -30px 0px -3px, rgb(255, 156, 85) 30px -30px, rgb(255, 255, 255) 40px -40px 0px -3px, rgb(255, 85, 85) 40px -40px; padding:20px; margin-right: 40px; font-size:30px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(60, 121, 245);"><b>Table of contents</b></div>

<div style="background-color: rgba(60, 121, 245, 0.03); padding:30px; font-size:15px; font-family: consolas;">
<ul>
    <li><a href="#1" target="_self" rel=" noreferrer nofollow">1. Setting Up Working Environment & Getting Started </a> </li>
    <li><a href="#2" target="_self" rel=" noreferrer nofollow">2. LLM Chain </a></li>
    <li><a href="#3" target="_self" rel=" noreferrer nofollow">3. Sequential Chains </a> 
        <ul>
<li><a href="#3.1" target="_self" rel=" noreferrer nofollow">3.1. Simple Sequential Chain</a></li>
<li><a href="#3.2" target="_self" rel=" noreferrer nofollow">3.2. Complex Sequential Chain</a></li>    
   </ul>
    </li>
    <li><a href="#4" target="_self" rel=" noreferrer nofollow">4. Router Chain </a></li> 
</ul>
</div>

***


<a id="1"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 1. Setting Up Working Environment & Getting Started </b></div>

We will start by loading the environment variables and the LLM that we will use in this article. Then we’re also going to load the data examples that we’re going to use.



In [1]:
!pip install langchain
!pip install langchain_community
!pip install openai

import os
import openai

from openai import OpenAI
import openai
import os
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
openai.api_key = user_secrets.get_secret("openai_api")
client = OpenAI(
    # This is the default and can be omitted
    api_key=openai.api_key,
)

llm_model = "gpt-3.5-turbo"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.8.2 requires keras

In [2]:
import pandas as pd
data_path =  '/kaggle/input/product-review/product review.csv'
df = pd.read_csv(data_path)
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


If we look inside this pandas data frame, we can see that there is a product column and then a review column. Each of these rows is a different data point that we can start passing through our chains.  

The first chain we’re going to cover is the LLM chain. This is a simple but really powerful chain that underpins a lot of the chains that we’ll go over in the future. 



<a id="2"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 2. LLM Chain </b></div>


We will start with importing three different things. We’re going to import the OpenAI model, the chat prompt, and the LLMChain. 


In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

Let's initialize the language model that we want to use. So we’re gonna initialize the chat open AI with a temperature, with a high temperature so that we can get diverse answers. 

In [4]:

llm = ChatOpenAI(temperature=0.9, model=llm_model, openai_api_key=openai.api_key)


/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Now we will initialize a prompt and this prompt is going to take in a variable called product. It’s gonna ask the LLM to generate the best name to describe a company that makes that product.

In [5]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

Finally, we’re going to combine these two things into a chain. This is what we call an LLM chain. And it’s quite simple. It’s just the combination of the LLM and the prompt. But now this chain will let us run through the prompt and into the LLM sequentially. 



In [6]:
chain = LLMChain(llm=llm, prompt=prompt)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


If we have a product called Deep Learning GPUs, we can run this through the chain by using chain.run(). This will do is it will format the prompt under the hood, and then it will pass the whole prompt into the LLM. We can see that we get back the name of this hypothetical company called **DeepMind GPUs**.

In [7]:
product = "Deep Learning GPUs"
chain.run(product)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'DeepMind GPU Technologies'

Here would be a good time to try it, you can input any product descriptions that you would want, and you can see what the chain will output as a result. 

So the LLM chain is the most basic type of chain, and that’s gonna be used a lot in the future. And so we can see how this will be used in the next type of chain, which will be sequential chains.

<a id="3"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 3. Sequential Chains </b></div>


Sequential chains run a sequence of chains, one after another. So to start, you’re going to import the simple sequential chain. This works well when we have subchains that expect only one input and return only one outpu

In [8]:
from langchain.chains import SimpleSequentialChain


<a id="3.1"></a>
## <div style="box-shadow: rgba(0, 0, 0, 0.18) 0px 2px 4px inset; padding:20px; font-size:24px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(67, 66, 66)"> <b> 3.1 Simple Sequential Chain </b></div>

We are going to first create one chain, which uses an LLM and a prompt. This prompt is going to take in the product and will return the best name to describe that company. So that will be the first chain.



In [9]:
llm = ChatOpenAI(temperature=0.9, model=llm_model, openai_api_key=openai.api_key)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)


Next, we’re going to create a second chain. The second chain will take in the company name and then output a 30-word description of that company. 



In [10]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 30 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

You can imagine how these chains might want to be run one after another, where the output of the first chain, the company name is then passed into the second chain. We can easily do this by creating a simple sequential chain where we have the two chains described there. We’ll call this overall simple chain. 

In [11]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True)

Now, we will run this chain over any product description. And so if we use it with the product above, the Deep Learning GPUs, we can run it over and we can see that it first outputs royal bedding, and then it passes it into the second chain and it comes up with this description of what that company could be about. 

In [12]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
NeuralTech Solutions
NeuralTech Solutions is a cutting-edge technology company specializing in artificial intelligence and machine learning solutions. We provide innovative and advanced technology to help businesses optimize their operations and improve efficiency.

> Finished chain.


'NeuralTech Solutions is a cutting-edge technology company specializing in artificial intelligence and machine learning solutions. We provide innovative and advanced technology to help businesses optimize their operations and improve efficiency.'

The simple sequential chain works well when there’s only a single input and a single output. But what about when there are multiple inputs or multiple outputs? And so we can do this by using just the regular sequential chain. 

<a id="3.2"></a>
## <div style="box-shadow: rgba(0, 0, 0, 0.18) 0px 2px 4px inset; padding:20px; font-size:24px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(67, 66, 66)"> <b> 3.2 Complex Sequential Chain </b></div>

So let’s import the sequential chain, and then you’re going to create a bunch of chains that we’re going to use one after another. 

In [13]:
from langchain.chains import SequentialChain

We’re going to be using the data from above, which has a review. In the first chain, we’re going to take the review and translate it into English. 

In [14]:
llm = ChatOpenAI(temperature=0.9, model=llm_model, openai_api_key=openai.api_key)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

With the second chain, we’re going to create a summary of that review in one sentence. This will use the previously generated English review.

In [15]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

The third chain is going to detect what the language of the review was in the first place. And so if you notice, this is using the review variable that is coming from the original review. 

In [16]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

Finally, the fourth chain will take in multiple inputs. So this will take in the summary variable, which we calculated with the second chain, and the language variable, which we calculated with the third chain. And it’s going to ask for a follow-up response to the summary in the specified language. 

In [17]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

One important thing to note about all these subchains is that the input keys and output keys need to be pretty precise. So here we’re taking in review. This is a variable that will be passed in at the start. We can see that we explicitly set the output key to English review. 

This is then used in the next prompt down below, where we take an English review with that same variable name. And we set the output key of that chain to the summary, which we can see is used in the final chain. 

The third prompt takes in review, the original variable, and outputs language, which is again used in the final prompt. It’s really important to get these variable names lined up exactly right because there are so many different inputs and outputs going on. And if you get any key errors, you should check that they are lined up. The simple sequential chain takes in multiple chains where each one has a single input and a single output. 

This is useful when you have more complicated downstream chains that need to be a composition of multiple previous chains. Now that we have all these chains, we can easily combine them into the sequential chain. So you’ll notice here that we’ll pass in the four chains we created into the chains variable. We’ll create the input variable with the one human input, which is the review. And then we want to return all of the intermediate outputs. So the English review, the summary, and then the follow-up message. 

In [18]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

Now we can run this over some of the data. So let’s choose a review and pass it in through the overall chain.

In [19]:
review = df.Review[5]
overall_chain(review)


/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer is disappointed in the taste of the product and suspects that it may be an old batch or counterfeit.',
 'followup_message': "Je suis désolé(e) d'apprendre que vous avez été déçu(e) par le goût du produit. Il est possible qu'il s'agisse effectivement d'un lot ancien ou contrefait. Pourriez-vous contacter le service client pour signaler votre expérience et obtenir des informations supplémentaires? Nous tenons à nous assurer de la qualité de nos produits et de la satisfaction de nos clients. Merci pour votre retour."}

We can see here that the original review looks like it was in French. We can see the English review as a translation. We can see a summary of that review, and then we can see a follow-up message in the original language of French.


<a id="4"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 4. Router Chain </b></div>


So far, we’ve covered the LLM chain and then a sequential chain. But what if you want to do something more complicated? A pretty common but basic operation is to route an input to a chain depending on what exactly that input is. 

A good way to imagine this is if you have multiple subchains, each of which is specialized for a particular type of input, you could have a router chain, which first decides which subchain to pass it to and then passes it to that chain. 

For a concrete example, let’s look at where we are routing between different types of chains depending on the subject that seems to come in. So we have here different prompts. One prompt is good for answering physics questions. The second prompt is good for answering math questions, the third for history, and then a fourth for computer science. Let’s define all these prompt templates. 

In [20]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

After we have these prompt templates, we can then provide more information about them. We can give each one a name and then a description. This information is going to be passed to the router chain, so the router chain can decide when to use this subchain. 

In [21]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

Let’s now import the other types of chains that we need. 

In [22]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

Here we need a multi-prompt chain. This is a specific type of chain that is used when routing between multiple different prompt templates. As you can see, all the options we have are prompt templates themselves. But this is just one type of thing that you can route between. You can route between any type of chain. 

The other classes that we’ll implement here are an LLM router chain. This uses a language model itself to route between the different subchains. This is where the description and the name provided above will be used. We’ll also import a router output parser. This parses the LLM output into a dictionary that can be used downstream to determine which chain to use and what the input to that chain should be. 

Now we can get around to using it. First, let’s import and define the language model that we will use. 

In [23]:
llm = ChatOpenAI(temperature=0, model=llm_model,openai_api_key=openai.api_key)

We now create the destination chains. These are the chains that will be called by the router chain. 



In [24]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)


As you can see, each destination chain itself is a language model chain, an LLM chain. In addition to the destination chains, we also need a default chain. This is the chain that’s called when the router can’t decide which of the sub-chains to use. For the example above, this might be called when the input question has nothing to do with physics, math, history, or computer science. 

In [25]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

Now we define the template that is used by the LLM to route between the different chains. This has instructions on the task to be done, as well as the specific formatting that the output should be in.

In [26]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

Let’s put a few of these pieces together to build the router chain. First, we create the full router template by formatting it with the destinations that we defined above. This template is flexible to a bunch of different types of destinations. 

In [27]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

Next, we create the prompt template from this template, and then we create the router chain by passing in the LLM and the overall router prompt. 

In [28]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

Note that here we have the router output parser. This is important as it will help this chain decide which subchains to route between. Finally, we will put it all together, we can create the overall chain. This has a router chain, which is defined here. It has destination chains, which we pass in here. Then we also pass in the default chain.

In [29]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

We can now use this chain. So let’s ask it some questions. We will ask it a physics question, we should hopefully see that it is routed to the physics chain with the input, what is blackbody radiation? And then that is passed into the chain down below, and we can see that the response is very detailed with lots of physics details. 

In [30]:
chain.run("What is Generative AI?")



> Entering new MultiPromptChain chain...
computer science: {'input': 'What is Generative AI in the field of computer science?'}
> Finished chain.


'Generative AI, also known as generative adversarial networks (GANs), is a type of artificial intelligence that is used to generate new data samples that resemble a given dataset. This technology involves two neural networks - a generator and a discriminator - that work together in a competitive manner to produce realistic outputs.\n\nThe generator network creates new data samples, such as images, music, or text, based on random noise input. The discriminator network then evaluates these generated samples and provides feedback to the generator on how realistic they are compared to the original dataset. Through this process of competition and feedback, the generator network learns to produce increasingly realistic outputs.\n\nGenerative AI has a wide range of applications, including image generation, text generation, and even deepfake technology. It is a powerful tool for creative tasks and can be used to generate new content, enhance existing data, or even create entirely new datasets 